<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/API_DART_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
pip install dart-fss

In [24]:
import requests
import pandas as pd

# DART API 키 입력
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 재무제표 요청 URL
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

In [25]:
def get_financial_statement(corp_code: str, year: str, fs_div: str = "CFS"):
    """
    DART API를 통해 재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2022)
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별)
    :return: DataFrame 형태의 재무제표 데이터
    """
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "110131",  # 11011: 사업보고서
        "fs_div": fs_div
    }

    # API 요청
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"API 오류: {data['message']}")
        return None

    # JSON 데이터를 DataFrame으로 변환
    reports = data['list']
    df = pd.DataFrame(reports)

    # 주요 컬럼 확인 및 데이터 정리
    expected_columns = ['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']
    available_columns = [col for col in expected_columns if col in df.columns]

    if not available_columns:
        print("유효한 재무제표 데이터가 없습니다.")
        return None

    df = df[available_columns]

    # 컬럼 이름 매핑
    column_mapping = {
        'sj_nm': '재무제표명',
        'account_nm': '계정명',
        'thstrm_amount': '당기 금액',
        'frmtrm_amount': '전기 금액',
        'bfefrmtrm_amount': '전전기 금액'
    }
    df.rename(columns=column_mapping, inplace=True)

    # 결측값 및 데이터 정제
    df.replace('', '0', inplace=True)
    numeric_columns = ['당기 금액', '전기 금액', '전전기 금액']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col])

    return df

# 예제 실행: 삼성전자(00126380) 2022년 재무제표 가져오기
corp_code = "00126380"  # DART에서 제공하는 회사 코드
year = "2023"
financial_data = get_financial_statement(corp_code, year)

if financial_data is not None:
    print(financial_data)
    # 엑셀로 저장
    financial_data.to_excel("financial_statement.xlsx", index=False)

     재무제표명                         계정명            당기 금액            전기 금액  \
0    재무상태표                        자산총계  455905980000000  448424507000000   
1    재무상태표                        유동자산  195936557000000  218470581000000   
2    재무상태표                         미수금    6633248000000    6149209000000   
3    재무상태표                        선급비용    3366130000000    2867823000000   
4    재무상태표                    현금및현금성자산   69080893000000   49680710000000   
..     ...                         ...              ...              ...   
171  자본변동표                   당기순이익(손실)   14473401000000   54730018000000   
172  자본변동표  관계기업 및 공동기업의 기타포괄손익에 대한 지분      75112000000     -50510000000   
173  자본변동표  관계기업 및 공동기업의 기타포괄손익에 대한 지분      70157000000     -51848000000   
174  자본변동표  관계기업 및 공동기업의 기타포괄손익에 대한 지분       4955000000       1338000000   
175  자본변동표  관계기업 및 공동기업의 기타포괄손익에 대한 지분      70157000000     -51848000000   

              전전기 금액  
0    426621158000000  
1    218163185000000  
2      44972570000

In [38]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_financial_statement(corp_code: str, year: str, fs_div: str = "OFS"):
    """
    DART API를 통해 개별재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2022)
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별)
    :return: DataFrame 형태의 재무제표 데이터
    """
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "110131",  # 사업보고서
        "fs_div": fs_div
    }

    # API 요청
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    # API 응답 상태 확인
    if data['status'] != '000':
        print(f"API 오류: {data['message']}")
        return None

    # 데이터 리스트 추출
    reports = data.get('list', [])
    if not reports:
        print("재무제표 데이터가 없습니다.")
        return None

    df = pd.DataFrame(reports)

    # API 응답 데이터 출력 (디버깅 용도)
    print("🔍 API 응답 데이터 일부 확인:")
    print(df.head())

    # 주요 컬럼 정리
    expected_columns = ['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']
    available_columns = [col for col in expected_columns if col in df.columns]

    if not available_columns:
        print("유효한 재무제표 데이터가 없습니다.")
        return None

    df = df[available_columns]

    # 컬럼 한글로 변경
    column_mapping = {
        'sj_nm': '재무제표명',
        'account_nm': '계정명',
        'thstrm_amount': '당기 금액',
        'frmtrm_amount': '전기 금액',
        'bfefrmtrm_amount': '전전기 금액'
    }
    df.rename(columns=column_mapping, inplace=True)

    # 금액 데이터 변환 (문자 → 숫자)
    numeric_columns = ['당기 금액', '전기 금액', '전전기 금액']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '', regex=True)  # 콤마 제거
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)  # 숫자로 변환

    return df

# 예제 실행: 삼성전자(00126380) 2023년 개별재무제표 가져오기
corp_code = "00126380"  # DART에서 제공하는 회사 코드
year = "2023"
fs_div = "OFS"  # 개별재무제표

financial_data = get_financial_statement(corp_code, year, fs_div)

if financial_data is not None:
    print(financial_data)
    # 엑셀 저장
    file_name = f"individual_financial_statement_{corp_code}_{year}.xlsx"
    financial_data.to_excel(file_name, index=False)
    print(f"📂 개별재무제표 데이터가 '{file_name}' 파일로 저장되었습니다.")


🔍 API 응답 데이터 일부 확인:
         rcept_no reprt_code bsns_year corp_code stock_code fs_div   fs_nm  \
0  20240312000736     110131      2023  00126380     005930    CFS  연결재무제표   
1  20240312000736     110131      2023  00126380     005930    CFS  연결재무제표   
2  20240312000736     110131      2023  00126380     005930    CFS  연결재무제표   
3  20240312000736     110131      2023  00126380     005930    CFS  연결재무제표   
4  20240312000736     110131      2023  00126380     005930    CFS  연결재무제표   

  sj_div  sj_nm account_nm  ...      thstrm_dt        thstrm_amount frmtrm_nm  \
0     BS  재무상태표       유동자산  ...  2023.12.31 현재  195,936,557,000,000    제 54 기   
1     BS  재무상태표      비유동자산  ...  2023.12.31 현재  259,969,423,000,000    제 54 기   
2     BS  재무상태표       자산총계  ...  2023.12.31 현재  455,905,980,000,000    제 54 기   
3     BS  재무상태표       유동부채  ...  2023.12.31 현재   75,719,452,000,000    제 54 기   
4     BS  재무상태표      비유동부채  ...  2023.12.31 현재   16,508,663,000,000    제 54 기   

       frmtrm_dt        

In [40]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_financial_statement(corp_code: str, year: str, fs_div_values: list = ["CFS", "OFS"]):
    """
    DART API를 통해 연결(CFS) 및 개별(OFS) 재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2023)
    :param fs_div_values: 재무제표 구분 리스트 (CFS: 연결, OFS: 개별)
    :return: DataFrame 형태의 통합 재무제표 데이터
    """
    all_data = []  # 데이터를 저장할 리스트

    for fs_div in fs_div_values:
        params = {
            "crtfc_key": API_KEY,
            "corp_code": corp_code,
            "bsns_year": year,
            "reprt_code": "11011",  # 사업보고서
            "fs_div": fs_div
        }

        # API 요청
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        # API 응답 상태 확인
        if data['status'] != '000':
            print(f"⚠️ API 오류 ({fs_div}): {data['message']}")
            continue

        # 데이터 리스트 추출
        reports = data.get('list', [])
        if not reports:
            print(f"⚠️ {fs_div} 재무제표 데이터가 없습니다.")
            continue

        df = pd.DataFrame(reports)

        # 주요 컬럼 정리
        expected_columns = ['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']
        available_columns = [col for col in expected_columns if col in df.columns]

        if not available_columns:
            print(f"⚠️ {fs_div} 재무제표에서 유효한 데이터가 없습니다.")
            continue

        df = df[available_columns]

        # 컬럼 한글로 변경
        column_mapping = {
            'sj_nm': '재무제표명',
            'account_nm': '계정명',
            'thstrm_amount': '당기 금액',
            'frmtrm_amount': '전기 금액',
            'bfefrmtrm_amount': '전전기 금액'
        }
        df.rename(columns=column_mapping, inplace=True)

        # 금액 데이터 변환 (문자 → 숫자)
        numeric_columns = ['당기 금액', '전기 금액', '전전기 금액']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].astype(str).str.replace(',', '', regex=True)  # 콤마 제거
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)  # 숫자로 변환

        # 재무제표 유형 추가 (CFS 또는 OFS)
        df['재무제표 구분'] = fs_div

        # 데이터 저장
        all_data.append(df)

    # 모든 데이터를 하나의 DataFrame으로 합치기
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        print("⚠️ 가져올 수 있는 재무제표 데이터가 없습니다.")
        return None

# 예제 실행: 삼성전자(00126380) 2023년 연결 및 개별 재무제표 가져오기
corp_code = "00126380"  # DART에서 제공하는 회사 코드
year = "2023"

financial_data = get_financial_statement(corp_code, year)

if financial_data is not None:
    print(financial_data)

    # 엑셀 저장
    file_name = f"financial_statement_{corp_code}_{year}.xlsx"
    financial_data.to_excel(file_name, index=False)
    print(f"📂 재무제표 데이터가 '{file_name}' 파일로 저장되었습니다.")


    재무제표명         계정명            당기 금액            전기 금액           전전기 금액  \
0   재무상태표        유동자산  195936557000000  218470581000000  218163185000000   
1   재무상태표       비유동자산  259969423000000  229953926000000  208457973000000   
2   재무상태표        자산총계  455905980000000  448424507000000  426621158000000   
3   재무상태표        유동부채   75719452000000   78344852000000   88117133000000   
4   재무상태표       비유동부채   16508663000000   15330051000000   33604094000000   
5   재무상태표        부채총계   92228115000000   93674903000000  121721227000000   
6   재무상태표         자본금     897514000000     897514000000     897514000000   
7   재무상태표       이익잉여금  346652238000000  337946407000000  293064763000000   
8   재무상태표        자본총계  363677865000000  354749604000000  304899931000000   
9   손익계산서         매출액  258935494000000  302231360000000  279604799000000   
10  손익계산서        영업이익    6566976000000   43376630000000   51633856000000   
11  손익계산서  법인세차감전 순이익   11006265000000   46440474000000   53351827000000   
12  손익계산서   

In [42]:
import requests
import pandas as pd
import json

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_financial_statement(corp_code: str, year: str, fs_div_values: list = ["CFS", "OFS"]):
    """
    DART API를 통해 연결(CFS) 및 개별(OFS) 재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2023)
    :param fs_div_values: 재무제표 구분 리스트 (CFS: 연결, OFS: 개별)
    :return: DataFrame 형태의 통합 재무제표 데이터
    """
    all_data = []  # 데이터를 저장할 리스트

    for fs_div in fs_div_values:
        params = {
            "crtfc_key": API_KEY,
            "corp_code": corp_code,
            "bsns_year": year,
            "reprt_code": "11012",  # 사업보고서
            "fs_div": fs_div
        }

        # API 요청
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        # API 응답 저장 (디버깅 용도)
        with open(f"data_{fs_div}.json", "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

        # API 응답 상태 확인
        if data['status'] != '000':
            print(f"⚠️ API 오류 ({fs_div}): {data['message']}")
            continue

        # 데이터 리스트 추출
        reports = data.get('list', [])
        if not reports:
            print(f"⚠️ {fs_div} 재무제표 데이터가 없습니다.")
            continue

        df = pd.DataFrame(reports)

        # 주요 컬럼 정리
        expected_columns = ['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']
        available_columns = [col for col in expected_columns if col in df.columns]

        if not available_columns:
            print(f"⚠️ {fs_div} 재무제표에서 유효한 데이터가 없습니다.")
            continue

        df = df[available_columns]

        # 컬럼 한글로 변경
        column_mapping = {
            'sj_nm': '재무제표명',
            'account_nm': '계정명',
            'thstrm_amount': '당기 금액',
            'frmtrm_amount': '전기 금액',
            'bfefrmtrm_amount': '전전기 금액'
        }
        df.rename(columns=column_mapping, inplace=True)

        # 금액 데이터 변환 (문자 → 숫자)
        numeric_columns = ['당기 금액', '전기 금액', '전전기 금액']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].astype(str).str.replace(',', '', regex=True)  # 콤마 제거
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)  # 숫자로 변환

        # 재무제표 유형 추가 (CFS 또는 OFS)
        df['재무제표 구분'] = fs_div

        # 데이터 저장
        all_data.append(df)

    # 모든 데이터를 하나의 DataFrame으로 합치기
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)

        # CFS와 OFS 비교를 위해 계정별 금액 차이 확인
        pivot_table = final_df.pivot_table(index="계정명", columns="재무제표 구분", values="당기 금액", aggfunc="sum").reset_index()
        print("\n📊 개별(OFS) vs 연결(CFS) 당기 금액 비교:")
        print(pivot_table.head(10))

        return final_df
    else:
        print("⚠️ 가져올 수 있는 재무제표 데이터가 없습니다.")
        return None

# 예제 실행: 삼성전자(00126380) 2023년 연결 및 개별 재무제표 가져오기
corp_code = "00126380"  # DART에서 제공하는 회사 코드
year = "2023"

financial_data = get_financial_statement(corp_code, year)

if financial_data is not None:
    print(financial_data)

    # 엑셀 저장
    file_name = f"financial_statement_{corp_code}_{year}.xlsx"
    financial_data.to_excel(file_name, index=False)
    print(f"📂 재무제표 데이터가 '{file_name}' 파일로 저장되었습니다.")



📊 개별(OFS) vs 연결(CFS) 당기 금액 비교:
재무제표 구분         계정명              CFS              OFS
0             당기순이익   11308204000000   11308204000000
1         당기순이익(손실)   11308204000000   11308204000000
2               매출액   99734621000000   99734621000000
3        법인세차감전 순이익   11181523000000   11181523000000
4              부채총계  156863727000000  156863727000000
5             비유동부채   47780533000000   47780533000000
6             비유동자산  456580279000000  456580279000000
7              영업이익   -3029556000000   -3029556000000
8              유동부채  109083194000000  109083194000000
9              유동자산  277447866000000  277447866000000
    재무제표명         계정명            당기 금액            전기 금액 재무제표 구분
0   재무상태표        유동자산  203975373000000  218470581000000     CFS
1   재무상태표       비유동자산  244025179000000  229953926000000     CFS
2   재무상태표        자산총계  448000552000000  448424507000000     CFS
3   재무상태표        유동부채   70780638000000   78344852000000     CFS
4   재무상태표       비유동부채   18244288000000   1533005100000

In [47]:
import json
import requests

# API 요청 함수
def get_api_data(corp_code, year, fs_div):
    API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
    BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11013",  # 사업보고서
        "fs_div": fs_div
    }

    response = requests.get(BASE_URL, params=params)
    return response.json()

# 삼성전자(00126380) 2023년 데이터 요청
corp_code = "00164742"
year = "2023"

data_OFS = get_api_data(corp_code, year, "OFS")
data_CFS = get_api_data(corp_code, year, "CFS")

# JSON 데이터 저장
with open("data_OFS.json", "w", encoding="utf-8") as f:
    json.dump(data_OFS, f, ensure_ascii=False, indent=4)

with open("data_CFS.json", "w", encoding="utf-8") as f:
    json.dump(data_CFS, f, ensure_ascii=False, indent=4)

print("✅ JSON 데이터 저장 완료. 'data_OFS.json'과 'data_CFS.json'을 직접 비교하세요.")


✅ JSON 데이터 저장 완료. 'data_OFS.json'과 'data_CFS.json'을 직접 비교하세요.


In [35]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_financial_statement(corp_code: str, year: str, fs_div: str = "OFS"):
    """
    DART API를 통해 개별재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2022)
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별)
    :return: DataFrame 형태의 재무제표 데이터
    """
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "110131",  # 사업보고서
        "fs_div": fs_div
    }

    # API 요청
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    # API 응답 상태 확인
    if data['status'] != '000':
        print(f"API 오류: {data['message']}")
        return None

    # API 응답 데이터 확인
    print("🔍 API 응답 데이터 일부 확인:")
    print(data)

    # 데이터 리스트 추출
    reports = data.get('list', [])
    if not reports:
        print("❌ 재무제표 데이터가 없습니다.")
        return None

    df = pd.DataFrame(reports)

    # 개별재무제표(`OFS`)만 필터링
    if 'fs_nm' in df.columns:
        df = df[df['fs_nm'].str.contains("개별", na=False)]

    if df.empty:
        print("❌ 개별재무제표 데이터가 없습니다.")
        return None

    # 주요 컬럼 정리
    expected_columns = ['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']
    available_columns = [col for col in expected_columns if col in df.columns]

    if not available_columns:
        print("⚠️ 유효한 재무제표 데이터가 없습니다.")
        return None

    df = df[available_columns]

    # 컬럼 한글로 변경
    column_mapping = {
        'sj_nm': '재무제표명',
        'account_nm': '계정명',
        'thstrm_amount': '당기 금액',
        'frmtrm_amount': '전기 금액',
        'bfefrmtrm_amount': '전전기 금액'
    }
    df.rename(columns=column_mapping, inplace=True)

    # 금액 데이터 변환 (문자 → 숫자)
    numeric_columns = ['당기 금액', '전기 금액', '전전기 금액']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '', regex=True)  # 콤마 제거
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)  # 숫자로 변환

    return df

# 예제 실행: 삼성전자(00126380) 2023년 개별재무제표 가져오기
corp_code = "00126380"  # DART에서 제공하는 회사 코드
year = "2023"
fs_div = "OFS"  # 개별재무제표

financial_data = get_financial_statement(corp_code, year, fs_div)

if financial_data is not None:
    print(financial_data)
    # 엑셀 저장
    file_name = f"individual_financial_statement_{corp_code}_{year}.xlsx"
    financial_data.to_excel(file_name, index=False)
    print(f"📂 개별재무제표 데이터가 '{file_name}' 파일로 저장되었습니다.")


🔍 API 응답 데이터 일부 확인:
{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20240312000736', 'reprt_code': '110131', 'bsns_year': '2023', 'corp_code': '00126380', 'stock_code': '005930', 'fs_div': 'CFS', 'fs_nm': '연결재무제표', 'sj_div': 'BS', 'sj_nm': '재무상태표', 'account_nm': '유동자산', 'thstrm_nm': '제 55 기', 'thstrm_dt': '2023.12.31 현재', 'thstrm_amount': '195,936,557,000,000', 'frmtrm_nm': '제 54 기', 'frmtrm_dt': '2022.12.31 현재', 'frmtrm_amount': '218,470,581,000,000', 'bfefrmtrm_nm': '제 53 기', 'bfefrmtrm_dt': '2021.12.31 현재', 'bfefrmtrm_amount': '218,163,185,000,000', 'ord': '1', 'currency': 'KRW'}, {'rcept_no': '20240312000736', 'reprt_code': '110131', 'bsns_year': '2023', 'corp_code': '00126380', 'stock_code': '005930', 'fs_div': 'CFS', 'fs_nm': '연결재무제표', 'sj_div': 'BS', 'sj_nm': '재무상태표', 'account_nm': '비유동자산', 'thstrm_nm': '제 55 기', 'thstrm_dt': '2023.12.31 현재', 'thstrm_amount': '259,969,423,000,000', 'frmtrm_nm': '제 54 기', 'frmtrm_dt': '2022.12.31 현재', 'frmtrm_amount': '229,953,926,000,0

In [14]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_financial_statement(corp_code: str, year: str, fs_div: str = "CFS", reprt_code: str = "110131"):
    """
    DART API를 통해 재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2022)
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별), 기본값: "CFS"
    :param reprt_code: 보고서 코드 (110131: 사업보고서, 11013: 3분기, 11012: 반기, 11014: 1분기)
    :return: DataFrame 형태의 재무제표 데이터
    """

    # API 요청 파라미터 설정
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": reprt_code,
        "fs_div": fs_div
    }

    try:
        # API 요청
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
        data = response.json()

        # API 응답 상태 확인
        if data.get('status') != '000':
            print(f"❌ API 오류: {data.get('message', '알 수 없는 오류')}")
            return None

        # 응답 데이터 확인
        reports = data.get('list', [])
        if not reports:
            print("❌ 재무제표 데이터가 없습니다.")
            return None

        # 데이터프레임 변환
        df = pd.DataFrame(reports)

        # 🔹 API에서 제공하는 컬럼 확인
        print("🔹 API 응답 컬럼 목록:", df.columns.tolist())

        # ✅ 계정명(account_nm) 출력
        if "account_nm" in df.columns:
            unique_accounts = df["account_nm"].unique()
            print("📌 총 계정명 목록:")
            for idx, acc in enumerate(unique_accounts, start=1):
                print(f"{idx}. {acc}")

        return df

    except requests.exceptions.RequestException as e:
        print(f"❌ API 요청 오류: {e}")
        return None
    except Exception as e:
        print(f"❌ 데이터 처리 오류: {e}")
        return None

if __name__ == "__main__":
    # 🔹 재무제표 조회 설정
    corp_code = "00126380"  # 삼성전자 DART 코드
    year = "2022"
    fs_div = "CFS"  # CFS(연결), OFS(개별)
    reprt_code = "110131"  # 사업보고서 (110131), 3분기(11013), 반기(11012), 1분기(11014)

    # 🔹 API 요청 및 데이터 가져오기
    financial_data = get_financial_statement(corp_code, year, fs_div, reprt_code)


🔹 API 응답 컬럼 목록: ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code', 'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm', 'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_nm', 'bfefrmtrm_dt', 'bfefrmtrm_amount', 'ord', 'currency']
📌 총 계정명 목록:
1. 유동자산
2. 비유동자산
3. 자산총계
4. 유동부채
5. 비유동부채
6. 부채총계
7. 자본금
8. 이익잉여금
9. 자본총계
10. 매출액
11. 영업이익
12. 법인세차감전 순이익
13. 당기순이익
14. 당기순이익(손실)


In [15]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def print_account_names(corp_code: str, year: str, fs_div: str = "CFS", reprt_code: str = "110131"):
    """
    DART API에서 특정 기업의 계정과목명(account_nm) 목록을 출력하는 함수
    """
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": reprt_code,
        "fs_div": fs_div
    }

    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # HTTP 오류 처리
        data = response.json()

        if data.get('status') != '000':
            print(f"❌ API 오류: {data.get('message', '알 수 없는 오류')}")
            return

        reports = data.get('list', [])
        if not reports:
            print("❌ 재무제표 데이터가 없습니다.")
            return

        df = pd.DataFrame(reports)

        # ✅ 계정과목명(account_nm)만 출력
        if "account_nm" in df.columns:
            unique_accounts = df["account_nm"].unique()
            print("📌 총 계정과목명 목록:")
            for idx, acc in enumerate(unique_accounts, start=1):
                print(f"{idx}. {acc}")
        else:
            print("❌ 'account_nm' 컬럼이 응답 데이터에 없습니다.")

    except requests.exceptions.RequestException as e:
        print(f"❌ API 요청 오류: {e}")
    except Exception as e:
        print(f"❌ 데이터 처리 오류: {e}")

if __name__ == "__main__":
    # 삼성전자(00126380) 2022년 사업보고서 조회
    corp_code = "00126380"
    year = "2022"
    print_account_names(corp_code, year)


📌 총 계정과목명 목록:
1. 유동자산
2. 비유동자산
3. 자산총계
4. 유동부채
5. 비유동부채
6. 부채총계
7. 자본금
8. 이익잉여금
9. 자본총계
10. 매출액
11. 영업이익
12. 법인세차감전 순이익
13. 당기순이익
14. 당기순이익(손실)


In [16]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def print_all_account_names(corp_code: str, year: str, fs_div: str = "CFS", reprt_code: str = "110131"):
    """
    DART API에서 모든 재무제표(재무상태표, 손익계산서 등)의 계정과목명을 출력하는 함수
    """
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": reprt_code,
        "fs_div": fs_div
    }

    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # HTTP 오류 처리
        data = response.json()

        if data.get('status') != '000':
            print(f"❌ API 오류: {data.get('message', '알 수 없는 오류')}")
            return

        reports = data.get('list', [])
        if not reports:
            print("❌ 재무제표 데이터가 없습니다.")
            return

        df = pd.DataFrame(reports)

        # ✅ 'sj_nm' (재무제표 종류)와 'account_nm' (계정과목명) 추출
        if "sj_nm" in df.columns and "account_nm" in df.columns:
            grouped = df.groupby("sj_nm")["account_nm"].unique()

            print("📌 모든 재무제표의 계정과목명 목록:")
            for statement, accounts in grouped.items():
                print(f"\n📄 [{statement}]")
                for idx, acc in enumerate(accounts, start=1):
                    print(f"  {idx}. {acc}")

        else:
            print("❌ 'sj_nm' 또는 'account_nm' 컬럼이 응답 데이터에 없습니다.")

    except requests.exceptions.RequestException as e:
        print(f"❌ API 요청 오류: {e}")
    except Exception as e:
        print(f"❌ 데이터 처리 오류: {e}")

if __name__ == "__main__":
    # 삼성전자(00126380) 2022년 사업보고서 조회
    corp_code = "00126380"
    year = "2022"
    print_all_account_names(corp_code, year)


📌 모든 재무제표의 계정과목명 목록:

📄 [손익계산서]
  1. 매출액
  2. 영업이익
  3. 법인세차감전 순이익
  4. 당기순이익
  5. 당기순이익(손실)

📄 [재무상태표]
  1. 유동자산
  2. 비유동자산
  3. 자산총계
  4. 유동부채
  5. 비유동부채
  6. 부채총계
  7. 자본금
  8. 이익잉여금
  9. 자본총계


In [17]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_all_account_details(corp_code: str, year: str, fs_div: str = "CFS"):
    """
    DART API에서 모든 재무제표(재무상태표, 손익계산서 등)의 계정과목 및 하위 계정과목을 포함하여 출력하는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: "2022")
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별), 기본값 "CFS"
    """

    # ✅ 보고서 코드 리스트 (사업보고서, 1분기, 반기, 3분기)
    reprt_codes = {
        "110131": "사업보고서",  # 연간 재무제표
        "11013": "3분기보고서",
        "11012": "반기보고서",
        "11014": "1분기보고서"
    }

    all_data = []

    for reprt_code, report_name in reprt_codes.items():
        params = {
            "crtfc_key": API_KEY,
            "corp_code": corp_code,
            "bsns_year": year,
            "reprt_code": reprt_code,
            "fs_div": fs_div
        }

        try:
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
            data = response.json()

            if data.get('status') != '000':
                print(f"❌ API 오류 ({report_name}): {data.get('message', '알 수 없는 오류')}")
                continue

            reports = data.get('list', [])
            if not reports:
                print(f"❌ {report_name} 재무제표 데이터가 없습니다.")
                continue

            df = pd.DataFrame(reports)

            # ✅ 필요한 컬럼만 추출 (계정명, 계정코드, 재무제표명 포함)
            if "sj_nm" in df.columns and "account_nm" in df.columns and "account_id" in df.columns:
                df_filtered = df[["sj_nm", "account_id", "account_nm"]].drop_duplicates()
                df_filtered["report_type"] = report_name  # 보고서 유형 추가
                all_data.append(df_filtered)

        except requests.exceptions.RequestException as e:
            print(f"❌ API 요청 오류 ({report_name}): {e}")
            continue
        except Exception as e:
            print(f"❌ 데이터 처리 오류 ({report_name}): {e}")
            continue

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

if __name__ == "__main__":
    # 삼성전자(00126380) 2022년 모든 재무제표 계정과목 조회
    corp_code = "00126380"
    year = "2022"
    fs_div = "CFS"  # 연결재무제표 (CFS) 또는 개별재무제표 (OFS)

    account_details = get_all_account_details(corp_code, year, fs_div)

    if account_details is not None:
        print(account_details)

        # ✅ 엑셀로 저장
        file_name = f"account_details_{corp_code}_{year}.xlsx"
        account_details.to_excel(file_name, index=False)
        print(f"✅ 계정과목 데이터가 '{file_name}' 파일로 저장되었습니다.")
    else:
        print("❌ 계정과목 데이터를 가져오지 못했습니다.")


❌ 계정과목 데이터를 가져오지 못했습니다.


In [19]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_all_account_details(corp_code: str, year: str, fs_div: str = "CFS"):
    """
    모든 재무제표의 계정과목을 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: "2022")
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별)
    :return: DataFrame (계정과목 목록)
    """
    # ✅ 사업보고서 코드 (DART API)
    reprt_codes = {
        "110131": "사업보고서",   # 연간
        "11013": "3분기보고서",
        "11012": "반기보고서",
        "11014": "1분기보고서"
    }

    all_data = []

    for reprt_code, report_name in reprt_codes.items():
        print(f"🔍 {report_name} 데이터 요청 중...")  # ✅ 진행상황 출력

        params = {
            "crtfc_key": API_KEY,
            "corp_code": corp_code,
            "bsns_year": year,
            "reprt_code": reprt_code,
            "fs_div": fs_div
        }

        try:
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
            data = response.json()

            print(f"✅ API 응답 상태 ({report_name}): {data.get('status', '알 수 없음')}")

            # 🔴 오류 코드 확인
            if data.get('status') != '000':
                print(f"❌ API 오류 ({report_name}): {data.get('message', '알 수 없는 오류')}")
                continue

            # 🔵 'list' 데이터 확인
            reports = data.get('list', [])
            if not reports:
                print(f"❌ {report_name} 데이터가 없습니다.")
                continue

            # ✅ DataFrame 변환
            df = pd.DataFrame(reports)

            # 🔹 계정과목(account_nm)만 추출
            if "sj_nm" in df.columns and "account_nm" in df.columns:
                df_filtered = df[["sj_nm", "account_nm"]].drop_duplicates()
                df_filtered["report_type"] = report_name  # 보고서 유형 추가
                all_data.append(df_filtered)

        except requests.exceptions.RequestException as e:
            print(f"❌ API 요청 오류 ({report_name}): {e}")
            continue
        except Exception as e:
            print(f"❌ 데이터 처리 오류 ({report_name}): {e}")
            continue

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        print("❌ 계정과목 데이터를 가져오지 못했습니다.")
        return None


# ✅ 실행 예제
if __name__ == "__main__":
    corp_code = "00126380"  # 삼성전자
    year = "2022"  # 최신 연도 확인
    fs_div = "CFS"

    account_details = get_all_account_details(corp_code, year, fs_div)

    if account_details is not None:
        print(account_details)
        file_name = f"account_details_{corp_code}_{year}.xlsx"
        account_details.to_excel(file_name, index=False)
        print(f"✅ 계정과목 데이터가 '{file_name}' 파일로 저장되었습니다.")
    else:
        print("❌ 계정과목 데이터를 가져오지 못했습니다.")


🔍 사업보고서 데이터 요청 중...
✅ API 응답 상태 (사업보고서): 000
🔍 3분기보고서 데이터 요청 중...
✅ API 응답 상태 (3분기보고서): 000
🔍 반기보고서 데이터 요청 중...
✅ API 응답 상태 (반기보고서): 000
🔍 1분기보고서 데이터 요청 중...
✅ API 응답 상태 (1분기보고서): 000
    sj_nm  account_nm report_type
0   재무상태표        유동자산       사업보고서
1   재무상태표       비유동자산       사업보고서
2   재무상태표        자산총계       사업보고서
3   재무상태표        유동부채       사업보고서
4   재무상태표       비유동부채       사업보고서
5   재무상태표        부채총계       사업보고서
6   재무상태표         자본금       사업보고서
7   재무상태표       이익잉여금       사업보고서
8   재무상태표        자본총계       사업보고서
9   손익계산서         매출액       사업보고서
10  손익계산서        영업이익       사업보고서
11  손익계산서  법인세차감전 순이익       사업보고서
12  손익계산서       당기순이익       사업보고서
13  손익계산서   당기순이익(손실)       사업보고서
14  재무상태표        유동자산      3분기보고서
15  재무상태표       비유동자산      3분기보고서
16  재무상태표        자산총계      3분기보고서
17  재무상태표        유동부채      3분기보고서
18  재무상태표       비유동부채      3분기보고서
19  재무상태표        부채총계      3분기보고서
20  재무상태표         자본금      3분기보고서
21  재무상태표       이익잉여금      3분기보고서
22  재무상태표        자본총계      3분기보고서


In [ ]:
# 손익계산서의 매출액, 매출원가, 판관비, 영업손익만 추출

# 재무제표 요청 URL
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_filtered_financial_statement(corp_code: str, year: str, fs_div: str = "CFS"):
    """
    손익계산서에서 매출액, 매출원가, 판관비, 영업손익만 추출하는 함수
    """
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 11011: 사업보고서
        "fs_div": fs_div
    }

    # API 요청
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"API 오류: {data['message']}")
        return None

    # JSON 데이터를 DataFrame으로 변환
    reports = data['list']
    df = pd.DataFrame(reports)

    # 손익계산서 항목 필터링 조건
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]

    # 필요한 컬럼만 추출
    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]

        # 컬럼 이름 매핑
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']

        # 결측값 및 데이터 정제
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric)

    return df

# 예제 실행: 삼성전자(00126380) 2022년 손익계산서 주요 항목 추출
corp_code = "00126380"  # DART에서 제공하는 회사 코드
year = "2022"
filtered_financial_data = get_filtered_financial_statement(corp_code, year)

if filtered_financial_data is not None:
    print(filtered_financial_data)
    # 엑셀로 저장
    filtered_financial_data.to_excel("income_statement_filtered.xlsx", index=False)

    재무제표명      계정명            당기 금액            전기 금액
54  손익계산서     매출원가  190041770000000  166411342000000
56  손익계산서  판매비와관리비   68812960000000   61559601000000
57  손익계산서     영업이익   43376630000000   51633856000000


In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os

# API 키 입력
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 기업 코드 요청 URL
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_corp_codes():
    """기업 코드 XML 파일 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    xml_path = "corpCode.xml"

    # XML 파일 저장
    with open(xml_path, "wb") as file:
        file.write(response.content)

    # XML 파싱
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # 외감 기업 코드 및 이름 추출
    corp_list = []
    for corp in root.findall('list'):
        corp_code = corp.find('corp_code').text
        corp_name = corp.find('corp_name').text
        stock_code = corp.find('stock_code').text

        # 외감기업(stock_code 없는 기업 포함)
        if stock_code is None or stock_code.strip() == "":
            corp_list.append((corp_code, corp_name))

    return corp_list

def get_financial_data(corp_code: str, year: str):
    """특정 기업의 재무제표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data['message']}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None

def collect_all_financials(year: str):
    """모든 외감 기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
        df = get_financial_data(corp_code, year)
        if df is not None:
            df['기업명'] = corp_name
            all_data.append(df)

    # 전체 데이터 통합
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"financial_statements_{year}.xlsx", index=False)
        print(f"{year}년도 외감기업 재무제표 수집 완료!")
    else:
        print("수집된 데이터가 없습니다.")

# 실행: 모든 외감기업의 2022년 재무제표 수집
collect_all_financials("2022")


ParseError: not well-formed (invalid token): line 1, column 2 (<string>)

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import io

# API 키 입력
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 기업 코드 요청 URL
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"

def download_corp_codes():
    """기업 코드 XML 파일 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)

    # 응답이 XML인지 확인
    if response.headers['Content-Type'] != 'application/xml':
        print("응답 형식이 XML이 아닙니다. 응답 내용:")
        print(response.text)
        return []

    # XML 파싱
    try:
        tree = ET.ElementTree(ET.fromstring(response.content))
        root = tree.getroot()

        # 외감 기업 코드 및 이름 추출
        corp_list = []
        for corp in root.findall('list'):
            corp_code = corp.find('corp_code').text
            corp_name = corp.find('corp_name').text
            stock_code = corp.find('stock_code').text

            # 외감기업 (stock_code 없는 기업 포함)
            if stock_code is None or stock_code.strip() == "":
                corp_list.append((corp_code, corp_name))

        return corp_list
    except ET.ParseError as e:
        print("XML 파싱 오류:", e)
        return []

# 테스트 실행
corp_list = download_corp_codes()
if corp_list:
    print(f"{len(corp_list)}개의 기업 코드가 수집되었습니다.")
else:
    print("기업 코드 수")


In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import io

# API 키 설정
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# API URL 설정
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_corp_codes():
    """외감기업 코드를 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)

    # 인코딩 설정 및 XML 읽기
    response.encoding = response.apparent_encoding
    xml_data = response.content.decode(response.encoding)

    # XML 파싱
    root = ET.fromstring(xml_data)

    # 외감 기업 코드 및 이름 추출
    corp_list = []
    for corp in root.findall('list'):
        corp_code = corp.find('corp_code').text
        corp_name = corp.find('corp_name').text
        stock_code = corp.find('stock_code').text

        # 외감 기업(stock_code 없는 기업 포함)
        if stock_code is None or stock_code.strip() == "":
            corp_list.append((corp_code, corp_name))

    return corp_list

def get_financial_data(corp_code: str, year: str):
    """특정 기업의 손익계산서 주요 항목 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data['message']}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None

def collect_all_financials(year: str):
    """모든 외감기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
        df = get_financial_data(corp_code, year)
        if df is not None:
            df['기업명'] = corp_name
            all_data.append(df)

    # 전체 데이터 통합
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"financial_statements_{year}.xlsx", index=False)
        print(f"{year}년도 외감기업 재무제표 수집 완료!")
    else:
        print("수집된 데이터가 없습니다.")

# 실행: 모든 외감기업의 2022년 재무제표 수집
collect_all_financials("2022")


TypeError: decode() argument 'encoding' must be str, not None

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# API 키 설정
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# API URL 설정
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_corp_codes():
    """외감기업 코드를 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)

    # 인코딩 설정: 'utf-8'을 기본으로 설정
    encoding = response.apparent_encoding if response.apparent_encoding else 'utf-8'
    xml_data = response.content.decode(encoding)

    # XML 파싱
    root = ET.fromstring(xml_data)

    # 외감 기업 코드 및 이름 추출
    corp_list = []
    for corp in root.findall('list'):
        corp_code = corp.find('corp_code').text
        corp_name = corp.find('corp_name').text
        stock_code = corp.find('stock_code').text

        # 외감기업 (stock_code 없는 기업 포함)
        if stock_code is None or stock_code.strip() == "":
            corp_list.append((corp_code, corp_name))

    return corp_list

def get_financial_data(corp_code: str, year: str):
    """특정 기업의 손익계산서 주요 항목 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data['message']}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None

def collect_all_financials(year: str):
    """모든 외감기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
        df = get_financial_data(corp_code, year)
        if df is not None:
            df['기업명'] = corp_name
            all_data.append(df)

    # 전체 데이터 통합
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"financial_statements_{year}.xlsx", index=False)
        print(f"{year}년도 외감기업 재무제표 수집 완료!")
    else:
        print("수집된 데이터가 없습니다.")

# 실행: 모든 외감기업의 2022년 재무제표 수집
collect_all_financials("2022")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8c in position 10: invalid start byte

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_corp_codes():
    """기업 코드 파일 다운로드 후 압축 해제 및 XML 파싱"""
    response = requests.get(CORP_CODE_URL)

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        # XML 파싱
        root = ET.fromstring(xml_data)

        # 외감 기업 코드 및 이름 추출
        corp_list = []
        for corp in root.findall('list'):
            corp_code = corp.find('corp_code').text
            corp_name = corp.find('corp_name').text
            stock_code = corp.find('stock_code').text

            # 외감기업 (stock_code 없는 기업 포함)
            if stock_code is None or stock_code.strip() == "":
                corp_list.append((corp_code, corp_name))

        return corp_list
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")
        return []

def get_financial_data(corp_code: str, year: str):
    """특정 기업의 손익계산서 주요 항목 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data['message']}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None

def collect_all_financials(year: str):
    """모든 외감기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
        df = get_financial_data(corp_code, year)
        if df is not None:
            df['기업명'] = corp_name
            all_data.append(df)

    # 전체 데이터 통합
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"financial_statements_{year}.xlsx", index=False)
        print(f"{year}년도 외감기업 재무제표 수집 완료!")
    else:
        print("수집된 데이터가 없습니다.")

# 실행: 모든 외감기업의 2022년 재무제표 수집
collect_all_financials("2022")


기업 다코(00434003) 재무제표 수집 중...
기업 코드 00434003 - API 오류: 조회된 데이타가 없습니다.
기업 굿앤엘에스(00430964) 재무제표 수집 중...
기업 코드 00430964 - API 오류: 조회된 데이타가 없습니다.
기업 크레디피아제이십오차유동화전문회사(00388953) 재무제표 수집 중...
기업 코드 00388953 - API 오류: 조회된 데이타가 없습니다.
기업 연방건설산업(00179984) 재무제표 수집 중...
기업 코드 00179984 - API 오류: 조회된 데이타가 없습니다.
기업 브룩스피알아이오토메이션잉크(00420143) 재무제표 수집 중...
기업 코드 00420143 - API 오류: 조회된 데이타가 없습니다.
기업 매경아이비아이(00401111) 재무제표 수집 중...
기업 코드 00401111 - API 오류: 조회된 데이타가 없습니다.
기업 캐드뱅크(00435534) 재무제표 수집 중...


KeyboardInterrupt: 

In [ ]:
corp_list

NameError: name 'corp_list' is not defined

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io
from concurrent.futures import ThreadPoolExecutor

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"


def download_corp_codes():
    """기업 코드 파일 다운로드 후 압축 해제 및 XML 파싱"""
    response = requests.get(CORP_CODE_URL)

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        # XML 파싱
        root = ET.fromstring(xml_data)
        corp_list = []
        for corp in root.findall('list'):
            corp_code = corp.find('corp_code').text
            corp_name = corp.find('corp_name').text
            stock_code = corp.find('stock_code').text

            # 외감기업 (stock_code 없는 기업 포함)
            if stock_code is None or stock_code.strip() == "":
                corp_list.append((corp_code, corp_name))

        return corp_list
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")
        return []


def get_financial_data(corp_code: str, year: str):
    """특정 기업의 손익계산서 주요 항목 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    try:
        data = response.json()
    except ValueError:
        print(f"기업 코드 {corp_code} - JSON 파싱 오류")
        return None

    if data.get('status') != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data.get('message', '알 수 없는 오류')}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None


def collect_financial_data_for_corp(corp_tuple, year):
    """특정 외감기업 데이터 수집"""
    corp_code, corp_name = corp_tuple
    print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
    df = get_financial_data(corp_code, year)
    if df is not None:
        df['기업명'] = corp_name
    return df


def collect_all_financials(year: str):
    """모든 외감기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()

    all_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        results =


SyntaxError: invalid syntax (<ipython-input-12-24237b243961>, line 94)

In [ ]:
import requests
import pandas as pd

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_financial_statement(corp_code: str, year: str):
    """특정 기업의 재무제표 데이터 다운로드 및 저장"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "OFS"  # 개별 재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000':
        reports = data.get('list', [])
        if reports:
            df = pd.DataFrame(reports)
            # 주요 컬럼 필터링
            df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)

            # 엑셀 저장
            filename = f"financial_statement_{corp_code}_{year}.xlsx"
            df.to_excel(filename, index=False)
            print(f"재무제표가 {filename}에 저장되었습니다.")
        else:
            print(f"기업 코드 {corp_code}에 대한 재무제표 데이터가 없습니다.")
    else:
        print(f"재무제표 다운로드 오류: {data.get('message', '알 수 없는 오류')}")

# 예제 실행: 특정 기업 2022년도 재무제표 다운로드

corp_code = "00126380"  # 기업 코드 입력
year = "2018"
download_financial_statement(corp_code, year)


재무제표가 financial_statement_00126380_2018.xlsx에 저장되었습니다.


In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_corp_codes():
    """기업 코드 리스트 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    corp_list = []

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        root = ET.fromstring(xml_data)
        for corp in root.findall("list"):
            corp_code = corp.find("corp_code").text
            corp_name = corp.find("corp_name").text
            stock_code = corp.find("stock_code").text
            if corp_code and corp_name:  # 외감 기업도 포함
                corp_list.append((corp_code, corp_name))
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")

    return corp_list

def get_financial_data(corp_code: str, corp_name: str, year: str):
    """특정 기업의 재무상태표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결 재무제표 (개별 재무제표는 OFS)
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000' and 'list' in data:
        df = pd.DataFrame(data['list'])
        # 재무상태표만 필터링
        df = df[df['sj_nm'].str.contains('재무상태표|대차대조표', na=False)]
        if not df.empty:
            df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
            df['기업명'] = corp_name
            return df
    return None

def collect_balance_sheets(year: str):
    """모든 기업의 재무상태표 데이터를 수집하여 엑셀로 저장"""
    corp_list = get_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무상태표 수집 중...")
        df = get_financial_data(corp_code, corp_name, year)
        if df is not None:
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = f"balance_sheets_{year}.xlsx"
        final_df.to_excel(filename, index=False)
        print(f"{filename}에 모든 재무상태표 저장 완료!")
    else:
        print("수집된 재무상태표 데이터가 없습니다.")

# 실행: 모든 기업의 2022년 재무상태표 수집
collect_balance_sheets("2022")


기업 다코(00434003) 재무상태표 수집 중...
기업 굿앤엘에스(00430964) 재무상태표 수집 중...
기업 크레디피아제이십오차유동화전문회사(00388953) 재무상태표 수집 중...
기업 연방건설산업(00179984) 재무상태표 수집 중...
기업 브룩스피알아이오토메이션잉크(00420143) 재무상태표 수집 중...
기업 매경아이비아이(00401111) 재무상태표 수집 중...
기업 캐드뱅크(00435534) 재무상태표 수집 중...
기업 엠와이오피삼차유동화전문유한회사(00430186) 재무상태표 수집 중...
기업 엠와이오피이차유동화전문유한회사(00430201) 재무상태표 수집 중...
기업 엠와이오피일차유동화전문유한회사(00430210) 재무상태표 수집 중...
기업 포스미디어(00430229) 재무상태표 수집 중...
기업 축복할렐루야(00140432) 재무상태표 수집 중...
기업 한국전자화학(00426208) 재무상태표 수집 중...
기업 ConnachtCapitalMarketInvestmentLtd.(00433262) 재무상태표 수집 중...
기업 선진아이티(00433749) 재무상태표 수집 중...
기업 팀스코리아(00433785) 재무상태표 수집 중...
기업 에넥스하이테크(00196079) 재무상태표 수집 중...
기업 세이스텝바이스텝혼합형펀드(00435048) 재무상태표 수집 중...
기업 유리알파헷지채권혼합형펀드(00435057) 재무상태표 수집 중...
기업 한기술정보통신(00108843) 재무상태표 수집 중...
기업 다움종합건설(00443232) 재무상태표 수집 중...
기업 한국애치슨(00248293) 재무상태표 수집 중...
기업 고은상사(00455662) 재무상태표 수집 중...
기업 조흥종합건설(00149318) 재무상태표 수집 중...
기업 유니즈유통(00451347) 재무상태표 수집 중...
기업 창대화장품(00415105) 재무상태표 수집 중...
기업 문화종합건설(00420824) 재무상태표 수집 중...
기업

KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_corp_codes():
    """기업 코드 리스트 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    corp_list = []

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        root = ET.fromstring(xml_data)
        for corp in root.findall("list"):
            corp_code = corp.find("corp_code").text
            corp_name = corp.find("corp_name").text
            if corp_code and corp_name:  # 외감 기업도 포함
                corp_list.append((corp_code, corp_name))
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")

    return corp_list

def get_financial_data(corp_code: str, corp_name: str, year: str):
    """특정 기업의 재무상태표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결 재무제표 (개별 재무제표는 OFS)
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000' and 'list' in data:
        df = pd.DataFrame(data['list'])
        # 재무상태표만 필터링
        df = df[df['sj_nm'].str.contains('재무상태표|대차대조표', na=False)]
        if not df.empty:
            df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
            df['기업명'] = corp_name
            df['기업코드'] = corp_code  # 기업코드 추가
            return df
    return None

def collect_balance_sheets(year: str):
    """모든 기업의 재무상태표 데이터를 수집하여 엑셀로 저장"""
    corp_list = get_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무상태표 수집 중...")
        df = get_financial_data(corp_code, corp_name, year)
        if df is not None:
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = f"balance_sheets_{year}.xlsx"
        final_df.to_excel(filename, index=False)
        print(f"{filename}에 모든 재무상태표 저장 완료!")
    else:
        print("수집된 재무상태표 데이터가 없습니다.")

# 실행: 모든 기업의 2022년 재무상태표 수집
collect_balance_sheets("2022")


기업 다코(00434003) 재무상태표 수집 중...
기업 굿앤엘에스(00430964) 재무상태표 수집 중...
기업 크레디피아제이십오차유동화전문회사(00388953) 재무상태표 수집 중...
기업 연방건설산업(00179984) 재무상태표 수집 중...
기업 브룩스피알아이오토메이션잉크(00420143) 재무상태표 수집 중...
기업 매경아이비아이(00401111) 재무상태표 수집 중...
기업 캐드뱅크(00435534) 재무상태표 수집 중...
기업 엠와이오피삼차유동화전문유한회사(00430186) 재무상태표 수집 중...
기업 엠와이오피이차유동화전문유한회사(00430201) 재무상태표 수집 중...
기업 엠와이오피일차유동화전문유한회사(00430210) 재무상태표 수집 중...
기업 포스미디어(00430229) 재무상태표 수집 중...
기업 축복할렐루야(00140432) 재무상태표 수집 중...
기업 한국전자화학(00426208) 재무상태표 수집 중...
기업 ConnachtCapitalMarketInvestmentLtd.(00433262) 재무상태표 수집 중...
기업 선진아이티(00433749) 재무상태표 수집 중...
기업 팀스코리아(00433785) 재무상태표 수집 중...
기업 에넥스하이테크(00196079) 재무상태표 수집 중...
기업 세이스텝바이스텝혼합형펀드(00435048) 재무상태표 수집 중...
기업 유리알파헷지채권혼합형펀드(00435057) 재무상태표 수집 중...
기업 한기술정보통신(00108843) 재무상태표 수집 중...
기업 다움종합건설(00443232) 재무상태표 수집 중...
기업 한국애치슨(00248293) 재무상태표 수집 중...
기업 고은상사(00455662) 재무상태표 수집 중...
기업 조흥종합건설(00149318) 재무상태표 수집 중...
기업 유니즈유통(00451347) 재무상태표 수집 중...
기업 창대화장품(00415105) 재무상태표 수집 중...
기업 문화종합건설(00420824) 재무상태표 수집 중...
기업

KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_corp_codes():
    """기업 코드 리스트 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    corp_list = []

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        root = ET.fromstring(xml_data)
        for corp in root.findall("list"):
            corp_code = corp.find("corp_code").text
            corp_name = corp.find("corp_name").text
            if corp_code and corp_name:  # 외감 기업도 포함
                corp_list.append((corp_code, corp_name))
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")

    return corp_list

def get_financial_data(corp_code: str, corp_name: str, year: str):
    """특정 기업의 재무상태표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결 재무제표 (개별 재무제표는 OFS)
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000' and 'list' in data:
        df = pd.DataFrame(data['list'])
        # 재무상태표만 필터링
        df = df[df['sj_nm'].str.contains('재무상태표|대차대조표', na=False)]
        if not df.empty:
            df = df[['sj_nm', 'account_id', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정코드', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
            df['기업명'] = corp_name
            df['기업코드'] = corp_code
            return df
    return None

def collect_balance_sheets(year: str):
    """모든 기업의 재무상태표 데이터를 수집하여 엑셀로 저장"""
    corp_list = get_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무상태표 수집 중...")
        df = get_financial_data(corp_code, corp_name, year)
        if df is not None:
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = f"balance_sheets_{year}.xlsx"
        final_df.to_excel(filename, index=False)
        print(f"{filename}에 모든 재무상태표 저장 완료!")
    else:
        print("수집된 재무상태표 데이터가 없습니다.")

# 실행: 모든 기업의 2022년 재무상태표 수집
collect_balance_sheets("2023")


기업 다코(00434003) 재무상태표 수집 중...
기업 굿앤엘에스(00430964) 재무상태표 수집 중...
기업 크레디피아제이십오차유동화전문회사(00388953) 재무상태표 수집 중...
기업 연방건설산업(00179984) 재무상태표 수집 중...
기업 브룩스피알아이오토메이션잉크(00420143) 재무상태표 수집 중...
기업 매경아이비아이(00401111) 재무상태표 수집 중...
기업 캐드뱅크(00435534) 재무상태표 수집 중...


KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_corp_codes():
    """기업 코드 리스트 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    corp_list = []

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        root = ET.fromstring(xml_data)
        for corp in root.findall("list"):
            corp_code = corp.find("corp_code").text
            corp_name = corp.find("corp_name").text
            if corp_code and corp_name:
                corp_list.append((corp_code, corp_name))
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")

    return corp_list

def get_financial_data(corp_code: str, corp_name: str, year: str):
    """특정 기업의 재무상태표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결 재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000' and 'list' in data:
        df = pd.DataFrame(data['list'])
        df = df[df['sj_nm'].str.contains('재무상태표|대차대조표', na=False)]
        if not df.empty:
            df = df[['sj_nm', 'account_id', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정코드', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
            df['기업명'] = corp_name
            df['기업코드'] = corp_code
            return df
    return None

def collect_balance_sheets(year: str):
    """모든 기업의 재무상태표 데이터를 수집하여 CSV로 저장"""
    corp_list = get_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무상태표 수집 중...")
        df = get_financial_data(corp_code, corp_name, year)
        if df is not None:
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = f"balance_sheets_{year}.csv"
        final_df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"{filename}에 모든 재무상태표 저장 완료!")
    else:
        print("수집된 재무상태표 데이터가 없습니다.")

# 실행: 모든 기업의 2022년 재무상태표 수집
collect_balance_sheets("2022")


기업 다코(00434003) 재무상태표 수집 중...
기업 굿앤엘에스(00430964) 재무상태표 수집 중...
기업 크레디피아제이십오차유동화전문회사(00388953) 재무상태표 수집 중...
기업 연방건설산업(00179984) 재무상태표 수집 중...
기업 브룩스피알아이오토메이션잉크(00420143) 재무상태표 수집 중...
기업 매경아이비아이(00401111) 재무상태표 수집 중...
기업 캐드뱅크(00435534) 재무상태표 수집 중...
기업 엠와이오피삼차유동화전문유한회사(00430186) 재무상태표 수집 중...
기업 엠와이오피이차유동화전문유한회사(00430201) 재무상태표 수집 중...
기업 엠와이오피일차유동화전문유한회사(00430210) 재무상태표 수집 중...
기업 포스미디어(00430229) 재무상태표 수집 중...
기업 축복할렐루야(00140432) 재무상태표 수집 중...
기업 한국전자화학(00426208) 재무상태표 수집 중...
기업 ConnachtCapitalMarketInvestmentLtd.(00433262) 재무상태표 수집 중...
기업 선진아이티(00433749) 재무상태표 수집 중...
기업 팀스코리아(00433785) 재무상태표 수집 중...
기업 에넥스하이테크(00196079) 재무상태표 수집 중...
기업 세이스텝바이스텝혼합형펀드(00435048) 재무상태표 수집 중...
기업 유리알파헷지채권혼합형펀드(00435057) 재무상태표 수집 중...
기업 한기술정보통신(00108843) 재무상태표 수집 중...
기업 다움종합건설(00443232) 재무상태표 수집 중...
기업 한국애치슨(00248293) 재무상태표 수집 중...
기업 고은상사(00455662) 재무상태표 수집 중...
기업 조흥종합건설(00149318) 재무상태표 수집 중...
기업 유니즈유통(00451347) 재무상태표 수집 중...
기업 창대화장품(00415105) 재무상태표 수집 중...
기업 문화종합건설(00420824) 재무상태표 수집 중...
기업

In [10]:
import requests
import pandas as pd

# DART API 키 및 기본 URL (외부에서 설정 필요)
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_financial_statement(corp_code: str, year: str, fs_div: str = "CFS", reprt_code: str = "110131"):
    """
    DART API를 통해 재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2022)
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별), 기본값: "CFS"
    :param reprt_code: 보고서 코드 (110131: 사업보고서, 11013: 3분기, 11012: 반기, 11014: 1분기)
    :return: DataFrame 형태의 재무제표 데이터
    """
    # fs_div 기본값 보정
    fs_div = fs_div.upper()
    if fs_div not in ["CFS", "OFS"]:
        print("잘못된 fs_div 값입니다. 기본값 'CFS'를 사용합니다.")
        fs_div = "CFS"

    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": reprt_code,
        "fs_div": fs_div
    }

    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
        data = response.json()

        if data['status'] != '000':
            print(f"API 오류: {data['message']}")
            return None

        # JSON 데이터를 DataFrame으로 변환
        reports = data.get('list', [])
        if not reports:
            print("재무제표 데이터가 없습니다.")
            return None

        df = pd.DataFrame(reports)

        # 주요 컬럼 확인 및 데이터 정리
        expected_columns = ['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']
        available_columns = [col for col in expected_columns if col in df.columns]

        if not available_columns:
            print("유효한 재무제표 데이터가 없습니다.")
            return None

        df = df[available_columns]

        # 컬럼 이름 매핑
        column_mapping = {
            'sj_nm': '재무제표명',
            'account_nm': '계정명',
            'thstrm_amount': '당기 금액',
            'frmtrm_amount': '전기 금액',
            'bfefrmtrm_amount': '전전기 금액'
        }
        df.rename(columns=column_mapping, inplace=True)

        # 데이터 정제 (결측값 처리 및 숫자 변환)
        df.replace({'': None}, inplace=True)  # 빈 문자열을 None으로 변환
        numeric_columns = ['당기 금액', '전기 금액', '전전기 금액']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

        return df

    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None

# 예제 실행: 삼성전자(00126380) 2022년 재무제표 가져오기
if __name__ == "__main__":
    corp_code = "00126380"  # DART에서 제공하는 회사 코드
    year = "2022"
    fs_div = "CFS"  # 연결재무제표 (CFS) 또는 개별재무제표 (OFS)
    reprt_code = "110131"  # 사업보고서 (110131)

    financial_data = get_financial_statement(corp_code, year, fs_div, reprt_code)

    if financial_data is not None:
        print(financial_data)
        # 엑셀로 저장
        file_name = f"financial_statement_{corp_code}_{year}.xlsx"
        financial_data.to_excel(file_name, index=False)
        print(f"재무제표 데이터가 '{file_name}' 파일로 저장되었습니다.")


    재무제표명         계정명  당기 금액  전기 금액  전전기 금액
0   재무상태표        유동자산    0.0    0.0     0.0
1   재무상태표       비유동자산    0.0    0.0     0.0
2   재무상태표        자산총계    0.0    0.0     0.0
3   재무상태표        유동부채    0.0    0.0     0.0
4   재무상태표       비유동부채    0.0    0.0     0.0
5   재무상태표        부채총계    0.0    0.0     0.0
6   재무상태표         자본금    0.0    0.0     0.0
7   재무상태표       이익잉여금    0.0    0.0     0.0
8   재무상태표        자본총계    0.0    0.0     0.0
9   손익계산서         매출액    0.0    0.0     0.0
10  손익계산서        영업이익    0.0    0.0     0.0
11  손익계산서  법인세차감전 순이익    0.0    0.0     0.0
12  손익계산서       당기순이익    0.0    0.0     0.0
13  손익계산서   당기순이익(손실)    0.0    0.0     0.0
14  재무상태표        유동자산    0.0    0.0     0.0
15  재무상태표       비유동자산    0.0    0.0     0.0
16  재무상태표        자산총계    0.0    0.0     0.0
17  재무상태표        유동부채    0.0    0.0     0.0
18  재무상태표       비유동부채    0.0    0.0     0.0
19  재무상태표        부채총계    0.0    0.0     0.0
20  재무상태표         자본금    0.0    0.0     0.0
21  재무상태표       이익잉여금    0.0    

In [7]:
import requests
import pandas as pd

# DART API 키 및 기본 URL
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

def get_financial_statement(corp_code: str, year: str, fs_div: str = "CFS", reprt_code: str = "110131"):
    """
    DART API를 통해 재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2022)
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별), 기본값: "CFS"
    :param reprt_code: 보고서 코드 (110131: 사업보고서, 11013: 3분기, 11012: 반기, 11014: 1분기)
    :return: DataFrame 형태의 재무제표 데이터
    """

    # API 요청 파라미터 설정
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": reprt_code,
        "fs_div": fs_div
    }

    try:
        # API 요청
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
        data = response.json()

        # API 응답 상태 확인
        if data.get('status') != '000':
            print(f"❌ API 오류: {data.get('message', '알 수 없는 오류')}")
            return None

        # 응답 데이터 확인
        reports = data.get('list', [])
        if not reports:
            print("❌ 재무제표 데이터가 없습니다.")
            return None

        # 데이터프레임 변환
        df = pd.DataFrame(reports)

        # 🔹 API에서 제공하는 컬럼 확인
        print("🔹 API 응답 컬럼 목록:", df.columns.tolist())

        # ✅ 존재하는 컬럼만 선택하여 사용
        column_mapping = {
            'sj_nm': '재무제표명',
            'account_nm': '계정명',
            'thstrm_amount': '당기 금액',
            'frmtrm_amount': '전기 금액',
            'bfefrmtrm_amount': '전전기 금액'
        }

        # 실제 존재하는 컬럼만 필터링
        existing_columns = {col: new_col for col, new_col in column_mapping.items() if col in df.columns}
        df = df[list(existing_columns.keys())].rename(columns=existing_columns)

        # 데이터 변환 (빈값 처리 및 숫자 변환)
        df.replace({'': None}, inplace=True)  # 빈 문자열을 None으로 변환
        for col in ['당기 금액', '전기 금액', '전전기 금액']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

        return df

    except requests.exceptions.RequestException as e:
        print(f"❌ API 요청 오류: {e}")
        return None
    except Exception as e:
        print(f"❌ 데이터 처리 오류: {e}")
        return None

if __name__ == "__main__":
    # 🔹 재무제표 조회 설정
    corp_code = "00126380"  # 삼성전자 DART 코드
    year = "2022"
    fs_div = "CFS"  # CFS(연결), OFS(개별)
    reprt_code = "110131"  # 사업보고서 (110131), 3분기(11013), 반기(11012), 1분기(11014)

    # 🔹 API 요청 및 데이터 가져오기
    financial_data = get_financial_statement(corp_code, year, fs_div, reprt_code)

    if financial_data is not None:
        # 데이터 출력
        print(financial_data)

        # 엑셀 파일로 저장
        file_name = f"financial_statement_{corp_code}_{year}.xlsx"
        financial_data.to_excel(file_name, index=False)
        print(f"✅ 재무제표 데이터가 '{file_name}' 파일로 저장되었습니다.")


🔹 API 응답 컬럼 목록: ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code', 'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm', 'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_nm', 'bfefrmtrm_dt', 'bfefrmtrm_amount', 'ord', 'currency']
    재무제표명         계정명  당기 금액  전기 금액  전전기 금액
0   재무상태표        유동자산    0.0    0.0     0.0
1   재무상태표       비유동자산    0.0    0.0     0.0
2   재무상태표        자산총계    0.0    0.0     0.0
3   재무상태표        유동부채    0.0    0.0     0.0
4   재무상태표       비유동부채    0.0    0.0     0.0
5   재무상태표        부채총계    0.0    0.0     0.0
6   재무상태표         자본금    0.0    0.0     0.0
7   재무상태표       이익잉여금    0.0    0.0     0.0
8   재무상태표        자본총계    0.0    0.0     0.0
9   손익계산서         매출액    0.0    0.0     0.0
10  손익계산서        영업이익    0.0    0.0     0.0
11  손익계산서  법인세차감전 순이익    0.0    0.0     0.0
12  손익계산서       당기순이익    0.0    0.0     0.0
13  손익계산서   당기순이익(손실)    0.0    0.0     0.0
14  재무상태표        유동자산    0.0    0.0     0.0
15  